# ILC NK HIPEC
Figure 2E and 2F

This script creates figures present in Figure 2 of the manuscript Marchalot et al. "Tumor-infiltrating immature innate lymphoid cells in colorectal cancer are biased towards tissue-resident NK cell/ILC1 differentiation"

Prerequisites: Load ILC_NK Seurat object available in GSE302045
Input: Seurat object
Output: plots as in Figure 2 in Marchalot et al.

In [ ]:
%pip install pandas==1.5.0

In [ ]:
from collections.abc import Iterable
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np
import pandas as pd
import anndata as ad
import os

In [ ]:
scv.settings.verbosity = 3
scv.settings.set_figure_params('scvelo', facecolor='white', dpi=100, frameon=False)
cr.settings.verbosity = 2

In [ ]:
output_dir = "Velocito_out/"
project_ID = "HIPEC"

In [ ]:
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
os.chdir(output_dir)

In [ ]:
# Load dataset
adata = sc.read_h5ad('path/to/ILC_NK.h5ad')


In [ ]:
# Load Loom files
ldata = scv.read("/path/to/all_samples_HIPEC.loom", cache=True)

In [ ]:
# Merge AnnData and Loom
adata = scv.utils.merge(adata, ldata)
print(adata)

In [ ]:
adata.obs['cell_type']  #Must be characters !

In [ ]:
# pre-process
#scv.pp.filter_and_normalize(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata, n_pcs=30, n_neighbors=30)
scv.pp.moments(adata, n_pcs=None, n_neighbors=None)

In [ ]:
# compute velocity
scv.tl.velocity(adata, mode='stochastic')
scv.tl.velocity_graph(adata)

## Figure 2E Velocity streams

In [ ]:
#Visualize velocity streams
scv.pl.velocity_embedding_stream(adata, basis='X_umap', color=['cell_type'], save='Figure_2E.pdf', title='By CellType',legend_loc='right margin')


# Figure 2F Velocity latent time

In [ ]:
# Calculate latent time
print("Computing latent time...")

# Method using recover_dynamics (recommended for latent time)
# This method fits the full transcriptional dynamics model
scv.tl.recover_dynamics(adata, n_jobs=1)

# Calculate latent time
scv.tl.latent_time(adata)

print("Latent time computation completed!")

In [ ]:
# Plot velocity embedding stream colored by latent time
scv.pl.velocity_embedding_stream(adata_March2023, basis='X_umap', color='latent_time', size=30, alpha=0.5,
                                 color_map='gnuplot', title='Velocity Stream - Latent Time',
                                 legend_loc='right margin', save='_Figure_2F.pdf')

# Save Anndata

In [ ]:
# Save final AnnData with velocities
adata.write_h5ad("path/to/save/directory/RNA_Velocity.h5ad")